In [5]:
import numpy as np 
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
import os
import itertools
os.environ['KMP_DUPLICATE_LIB_OK']='True'
plt.style.use('seaborn')
pd.options.display.max_rows = 4000
%matplotlib inline
pd.set_option('display.max_columns', 100)

Cleaning Up Data

In [30]:
df_NBA = pd.read_csv('1985_2000_draft_class_NBA.csv',index_col=0)
df_NCAA= pd.read_csv('NCAA_Stats.csv',index_col=0)

In [31]:
df_NCAA.fillna(0, inplace=True)
df_NBA.fillna(0, inplace=True)

In [32]:
df_NCAA['2P']= np.where(df_NCAA['2P']==0, df_NCAA['FG'], df_NCAA['2P'])
df_NCAA['2PA']= np.where(df_NCAA['2PA']==0, df_NCAA['FGA'], df_NCAA['2PA'])
df_NCAA['2P%']= np.where(df_NCAA['2P%']==0, df_NCAA['FG%'], df_NCAA['2P%'])

Combining Data for EDA, Feature Creation and Modeling

In [34]:
columns2drop= ['Pick', 'Team', 'College', 'G', 'MP', 'PTS', 'TRB',
       'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'WS',
       'WS/48', 'BPM', 'VORP', 'Position3']
df_NBA.drop(columns= columns2drop, inplace= True)

,Name,Yrs,Year,Height,Weight,Birthday,Position1,Position2,All_Star_apps,All_NBA_apps,All_Def_apps,HOF
0,Patrick Ewing,17.0,1985,213cm,108kg,1962-08-05,Center,Power Forward,11.0,7.0,3.0,Hall of Fame
1,Wayman Tisdale,12.0,1985,206cm,108kg,1964-06-09,Power Forward,Power Forward,0.0,0.0,0.0,0
2,Benoit Benjamin,15.0,1985,213cm,113kg,1964-11-22,Center,Center,0.0,0.0,0.0,0
3,Xavier McDaniel,12.0,1985,201cm,92kg,1963-06-04,Power Forward,Small Forward,1.0,0.0,0.0,0
4,Jon Koncak,11.0,1985,213cm,113kg,1963-05-17,Center,Center,0.0,0.0,0.0,0
5,Joe Kleine,15.0,1985,211cm,115kg,1962-01-04,Center,Center,0.0,0.0,0.0,0
6,Chris Mullin,16.0,1985,198cm,90kg,1963-07-30,Small Forward,Shooting Guard,5.0,4.0,0.0,Hall of Fame
7,Detlef Schrempf,16.0,1985,208cm,106kg,1963-01-21,Power Forward,Small Forward,3.0,0.0,0.0,0
8,Charles Oakley,19.0,1985,203cm,102kg,1963-12-18,Center,Power Forward,1.0,0.0,2.0,0
9,Ed Pinckney,12.0,1985,206cm,88kg,1963-03-27,Power Forward,Small Forward,0.0,0.0,0.0,0


In [39]:
df = df_NCAA.merge(df_NBA, on='Name', how='inner')